In [3]:
!python -m pip install pytesseract
!python -m pip install opencv-python

In [3]:
pip install pymupdf pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # replace this path with the path to your tesseract installation

In [2]:
import fitz
from PIL import Image
import pandas as pd
from pandas import DataFrame as df
from io import StringIO
from pathlib import Path
import csv

In [6]:
def pdf_to_png(pdf_path, output_folder):
    '''Chop a given PDF into individual pages, then convert each PDF into an image (saved to the pngs folder). Convert OCR data about each page into a .tsv DataFrame.
        This function requires that you have a folder in the same level as your pdf for outputs, and then two folders within that folder titled 'pngs' and 'tsv_data'.
        For example:
            your pdf
            output folder
                tsv_data
                pngs'''
    
    open_pdf = fitz.open(pdf_path)

    sum_string = ''
    tsv_total = pd.DataFrame()
    for page_num in range(open_pdf.page_count): # iterate through individual pages
        page = open_pdf[page_num]

        img = page.get_pixmap()
        

        # make the image
        pil_img = Image.frombytes("RGB", [img.width, img.height], img.samples) # convert to PIL Image
        # improve resolution
        scale_factor = 3.5 # try changing this to improve resolution
        new_size = (img.width * scale_factor, img.height * scale_factor)
        resize = pil_img.resize(new_size, Image.LANCZOS)
        

        # process image into tsv and clean some values
        png_to_data = pytesseract.image_to_data(resize, config = r'--psm 6') 
        data_to_tsv = StringIO(png_to_data)
        data_read = pd.read_csv(data_to_tsv, sep='\t', quoting=csv.QUOTE_NONE)
        tsv_clean = data_read[['line_num', 'word_num', 'left', 'top', 'text', 'conf']]
        tsv_total = pd.concat([tsv_total, tsv_clean])

        img_to_string = pytesseract.image_to_string(resize, config = r'--psm 6')
        sum_string += img_to_string

        # save to files
        output_csv_path = Path(f"{output_folder}/tsv_data/page_{page_num + 1}.tsv")
        #tsv_clean.to_csv(output_csv_path, sep = '\t', index = False)

        output_png_path = Path(f"{output_folder}/pngs/page_{page_num + 1}.png")
        resize.save(output_png_path)

    output_tsv_total = Path(f"{output_folder}/tsv_data.tsv")
    tsv_total.to_csv(output_tsv_total, sep= '\t', index=False)
    print(sum_string)
    output_str_path = Path(f"{output_folder}/str_data.txt")
    with open(output_str_path, 'w') as file:
        file.write(sum_string)

In [5]:
# replace these with the paths to your files
pdf_path = r'TOCs\NAFAM 3\NAFAM.3.2014_v1_cropped.pdf'
output_folder = r'TOCs\NAFAM 3\processed_v1'

pdf_to_png(pdf_path, output_folder)

Contents
a er
Volume |: Beginnings Through the Harlem Renaissance
PREPACE xxit
ACKNOWLEDGMENTS xxxi
INTRODUCTION: TALKING BOOKS xu
The Vernacular Tradition, Part |
INTRODUCTION 3
SpPiriTuats w
City Called Heaven * 12
| Know Moon-Rise 13
Evchicl Saw de Wheel © 13
I'm a-Rollin’ © 14
Go Down, Moses * 14
Been in the Storm So Long ® 15 dite
Swing Low, Sweet Chariot © 16
Steal Away to Jesus © 16
Didn't My Lord Deliver Danicl? ¢ 17
God's a Gonna Trouble the Water * 18
Soon | Will Be Done ¢ 18
Come Sunday * 19
Secucar Ruymes ano Soncs x» :
(We ruise de wheat} 21
Me and My Captain 21
Promises of Freedom 22
No More Auction Block * 23
Jack and Dinah Want Freedom 23
Run, Nigger, Run 24
[¢ indiates audio veloc tion avwileble om the SiudySpace}
ta
- » CONTENTS
Another Man Done Gone * 24
You May Go But This Will Bring You Back © 25
BALLADS 25
John Henry * 25
Frankie and Johnny * 28
Railroad Bill 29
The Signifying Monkey * 30
Stackolee * 32
Sinking of the Titanic 33
Shine and the Titanic 34
Work Songs

In [10]:
df1 = pd.DataFrame([1, 2, 3])
df2 = pd.DataFrame([1, 2, 3])
pd.concat([df1, df2])

,0
0,1
1,2
2,3
0,1
1,2
2,3
